In [1]:
# Cell 1: Imports & Global Config

import os
import glob
import random
from collections import defaultdict

import cv2
import numpy as np
from sklearn.model_selection import KFold
import torch
from ultralytics import YOLO

# === 你的資料根目錄 (請依照你的實際狀況調整) ===
BASE_DIR = r"C:\Users\User\Desktop\AICUP"

# 原始 CT 影像 & 標註 (有 patient 子資料夾)
SRC_IMG_ROOT = os.path.join(BASE_DIR, "42_training_image", "training_image")
SRC_LBL_ROOT = os.path.join(BASE_DIR, "42_training_label", "training_label")

# 產生 ROI 後的扁平 YOLO dataset
ROI_IMG_ROOT = os.path.join(BASE_DIR, "roi_dataset", "images")
ROI_LBL_ROOT = os.path.join(BASE_DIR, "roi_dataset", "labels")

# ROI 後再做 k-fold 的 YOLO 資料夾
KFOLD_ROOT = os.path.join(BASE_DIR, "yolo_kfold_roi")

os.makedirs(ROI_IMG_ROOT, exist_ok=True)
os.makedirs(ROI_LBL_ROOT, exist_ok=True)
os.makedirs(KFOLD_ROOT, exist_ok=True)

# 影像尺寸 (原始 CT 是 512x512)
IMG_W = 512
IMG_H = 512

# === ROI 設定 (基於你剛剛算出來 + 稍微多保守一點的版本) ===
# 這組 ROI 已經驗證過，能完全涵蓋全部 GT box：
# x1_min ≈ 130, y1_min ≈ 183, x2_max ≈ 287, y2_max ≈ 352
# 這裡再放寬一點點
ROI_X1 = 100
ROI_Y1 = 153
ROI_X2 = 317
ROI_Y2 = 382

ROI_W = ROI_X2 - ROI_X1
ROI_H = ROI_Y2 - ROI_Y1

print("ROI:", ROI_X1, ROI_Y1, "→", ROI_X2, ROI_Y2, "(W,H)=(", ROI_W, ",", ROI_H, ")")

# === K-fold 設定 ===
N_FOLDS = 5
SEED = 114514

# === YOLO 訓練設定 ===
MODEL_NAME = "yolo12s.pt"  # 放在當前工作目錄或給絕對路徑都可
IMG_SIZE = 512
EPOCHS = 45
BATCH_SIZE = 64
PROJECT = os.path.join(BASE_DIR, "runs", "detect_kfold_roi")

os.makedirs(PROJECT, exist_ok=True)
print("PROJECT:", PROJECT)


ROI: 100 153 → 317 382 (W,H)=( 217 , 229 )
PROJECT: C:\Users\User\Desktop\AICUP\runs\detect_kfold_roi


In [2]:
# Cell 2: 工具函式

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def yolo_to_pixel(cx, cy, w, h, img_w, img_h):
    """YOLO normalized -> pixel box (x1,y1,x2,y2)"""
    cx_px = cx * img_w
    cy_px = cy * img_h
    w_px = w * img_w
    h_px = h * img_h
    x1 = cx_px - w_px / 2
    y1 = cy_px - h_px / 2
    x2 = cx_px + w_px / 2
    y2 = cy_px + h_px / 2
    return x1, y1, x2, y2

def pixel_to_yolo(x1, y1, x2, y2, img_w, img_h):
    """pixel box (x1,y1,x2,y2) -> YOLO normalized (cx,cy,w,h)"""
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w / 2
    cy = y1 + h / 2
    return cx / img_w, cy / img_h, w / img_w, h / img_h

def get_patient_id_from_path(path: str) -> str:
    """
    假設檔名格式為: patientXXXX_YYYY.png / .txt
    例如 patient0075_0244.png -> patient0075
    """
    base = os.path.basename(path)
    name = os.path.splitext(base)[0]
    # name 形如: patient0075_0244
    return name.split("_")[0]


In [2]:
# Cell 3: 為所有圖片補空白 label

def ensure_all_images_have_labels(src_img_root, src_lbl_root):
    """
    在 SRC_LBL_ROOT 中，為每張 PNG 圖片補上一個空白 txt（如果沒有的話）
    """
    cnt_new = 0
    cnt_total = 0

    patient_dirs = sorted(
        d for d in os.listdir(src_img_root)
        if os.path.isdir(os.path.join(src_img_root, d))
    )

    for pd in patient_dirs:
        img_dir = os.path.join(src_img_root, pd)
        lbl_dir = os.path.join(src_lbl_root, pd)
        os.makedirs(lbl_dir, exist_ok=True)

        img_paths = sorted(glob.glob(os.path.join(img_dir, "*.png")))
        for img_path in img_paths:
            cnt_total += 1
            base = os.path.splitext(os.path.basename(img_path))[0]
            lbl_path = os.path.join(lbl_dir, base + ".txt")
            if not os.path.exists(lbl_path):
                # 建一個空白 label
                open(lbl_path, "w", encoding="utf-8").close()
                cnt_new += 1

    print(f"總共有圖片 {cnt_total} 張")
    print(f"補上新的空白 label 檔案數：{cnt_new}")

seed_everything(SEED)
ensure_all_images_have_labels(SRC_IMG_ROOT, SRC_LBL_ROOT)


NameError: name 'seed_everything' is not defined

In [4]:
# Cell 4: 產生 ROI YOLO dataset

def process_one_pair_to_roi(img_path, lbl_path):
    """對一張圖 + label 做 ROI crop 並輸出到 ROI_IMG_ROOT / ROI_LBL_ROOT"""
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"⚠ 無法讀圖：{img_path}")
        return 0

    h, w = img.shape[:2]
    if (h, w) != (IMG_H, IMG_W):
        print(f"⚠ 圖片尺寸不是 {IMG_W}x{IMG_H}：{img_path}（實際 {w}x{h}），仍然繼續，但請確認")

    # 1) 裁 ROI
    x1, y1, x2, y2 = ROI_X1, ROI_Y1, ROI_X2, ROI_Y2
    roi_img = img[y1:y2, x1:x2]

    base = os.path.splitext(os.path.basename(img_path))[0]
    dst_img_path = os.path.join(ROI_IMG_ROOT, base + ".png")
    dst_lbl_path = os.path.join(ROI_LBL_ROOT, base + ".txt")

    # 2) 寫 ROI 圖片
    cv2.imwrite(dst_img_path, roi_img)

    # 3) 讀 label（允許空白）
    if not os.path.exists(lbl_path):
        # 理論上前面已經補空了，這裡還是防呆一次
        open(dst_lbl_path, "w", encoding="utf-8").close()
        return 0

    with open(lbl_path, "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f.readlines() if ln.strip()]

    if not lines:
        # 純背景，寫空白 label
        open(dst_lbl_path, "w", encoding="utf-8").close()
        return 0

    new_lines = []
    dropped = 0
    for ln in lines:
        parts = ln.split()
        if len(parts) != 5:
            print(f"⚠ 非 YOLO 格式，略過: {lbl_path} -> {ln}")
            continue

        cls_id = int(float(parts[0]))
        cx_n, cy_n, w_n, h_n = map(float, parts[1:])

        # 原圖 normalized -> pixel
        x1_px, y1_px, x2_px, y2_px = yolo_to_pixel(cx_n, cy_n, w_n, h_n, IMG_W, IMG_H)

        # 平移到 ROI 座標系
        x1_roi = x1_px - ROI_X1
        y1_roi = y1_px - ROI_Y1
        x2_roi = x2_px - ROI_X1
        y2_roi = y2_px - ROI_Y1

        # clip 到 ROI 內
        x1_roi = max(0, min(ROI_W, x1_roi))
        y1_roi = max(0, min(ROI_H, y1_roi))
        x2_roi = max(0, min(ROI_W, x2_roi))
        y2_roi = max(0, min(ROI_H, y2_roi))

        if x2_roi <= x1_roi or y2_roi <= y1_roi:
            dropped += 1
            continue

        # ROI pixel -> normalized（以 ROI_W, ROI_H 基準）
        cx_roi_n, cy_roi_n, w_roi_n, h_roi_n = pixel_to_yolo(
            x1_roi, y1_roi, x2_roi, y2_roi, ROI_W, ROI_H
        )

        cx_roi_n = min(max(cx_roi_n, 0.0), 1.0)
        cy_roi_n = min(max(cy_roi_n, 0.0), 1.0)
        w_roi_n  = min(max(w_roi_n,  0.0), 1.0)
        h_roi_n  = min(max(h_roi_n,  0.0), 1.0)

        new_lines.append(
            f"{cls_id} {cx_roi_n:.6f} {cy_roi_n:.6f} {w_roi_n:.6f} {h_roi_n:.6f}"
        )

    with open(dst_lbl_path, "w", encoding="utf-8") as f:
        for nl in new_lines:
            f.write(nl + "\n")

    return len(new_lines)

def build_roi_dataset():
    total_imgs = 0
    total_boxes = 0

    patient_dirs = sorted(
        d for d in os.listdir(SRC_IMG_ROOT)
        if os.path.isdir(os.path.join(SRC_IMG_ROOT, d))
    )

    for pd in patient_dirs:
        img_dir = os.path.join(SRC_IMG_ROOT, pd)
        lbl_dir = os.path.join(SRC_LBL_ROOT, pd)
        img_paths = sorted(glob.glob(os.path.join(img_dir, "*.png")))
        print(f"📁 處理 {pd}，共有 {len(img_paths)} 張圖")

        for img_path in img_paths:
            base = os.path.splitext(os.path.basename(img_path))[0]
            lbl_path = os.path.join(lbl_dir, base + ".txt")
            n_box = process_one_pair_to_roi(img_path, lbl_path)
            total_imgs += 1
            total_boxes += n_box

    print("\n🎉 ROI dataset 產生完成！")
    print(f"總共處理圖片數：{total_imgs}")
    print(f"總共保留 bbox 數：{total_boxes}")
    print(f"ROI 圖片輸出路徑：{ROI_IMG_ROOT}")
    print(f"ROI label 輸出路徑：{ROI_LBL_ROOT}")

build_roi_dataset()


📁 處理 patient0001，共有 341 張圖


📁 處理 patient0002，共有 391 張圖
📁 處理 patient0003，共有 324 張圖
📁 處理 patient0004，共有 365 張圖


KeyboardInterrupt: 

In [ ]:
# Cell 5: 從 ROI dataset 產生 patient-wise k-fold YOLO 資料夾

def collect_roi_by_patient():
    """
    掃描 ROI_LBL_ROOT，建立:
      patient_to_files[pid] = [ (img_path, lbl_path), ... ]
    """
    patient_to_files = defaultdict(list)

    lbl_paths = sorted(glob.glob(os.path.join(ROI_LBL_ROOT, "*.txt")))
    print(f"在 ROI labels 找到 {len(lbl_paths)} 個標註檔")

    for lbl_path in lbl_paths:
        base = os.path.splitext(os.path.basename(lbl_path))[0]
        img_path = os.path.join(ROI_IMG_ROOT, base + ".png")
        if not os.path.exists(img_path):
            print(f"⚠ 找不到對應圖片：{img_path}，略過")
            continue

        pid = get_patient_id_from_path(lbl_path)
        patient_to_files[pid].append((img_path, lbl_path))

    print(f"共收集到 {len(patient_to_files)} 個病人")
    return patient_to_files

patient_to_files = collect_roi_by_patient()

# 確認一下每個病人的樣本數大概長什麼樣
for pid, files in list(patient_to_files.items())[:5]:
    print(pid, "樣本數 =", len(files))


在 ROI labels 找到 16863 個標註檔
共收集到 50 個病人
patient0001 樣本數 = 341
patient0002 樣本數 = 391
patient0003 樣本數 = 324
patient0004 樣本數 = 365
patient0005 樣本數 = 285


In [3]:
# Cell 6: 產生 N_FOLDS 個 fold 資料夾 + data.yaml

def write_list_to_yolo_folder(file_list, dest_img_dir, dest_lbl_dir):
    os.makedirs(dest_img_dir, exist_ok=True)
    os.makedirs(dest_lbl_dir, exist_ok=True)

    for img_src, lbl_src in file_list:
        base = os.path.basename(img_src)
        lbl_base = os.path.basename(lbl_src)

        img_dst = os.path.join(dest_img_dir, base)
        lbl_dst = os.path.join(dest_lbl_dir, lbl_base)

        # 直接 copy 檔案內容（CT 檔案不大，這樣最簡單）
        img = cv2.imread(img_src, cv2.IMREAD_UNCHANGED)
        cv2.imwrite(img_dst, img)

        with open(lbl_src, "r", encoding="utf-8") as f_in, \
             open(lbl_dst, "w", encoding="utf-8") as f_out:
            f_out.write(f_in.read())

def create_fold_datasets(n_folds=5, seed=42):
    patients = sorted(patient_to_files.keys())
    print("病人數量:", len(patients))

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(patients), start=1):
        train_pids = [patients[i] for i in train_idx]
        val_pids   = [patients[i] for i in val_idx]

        print(f"\n=== Fold {fold_idx} ===")
        print("Train 病人數:", len(train_pids))
        print("Val   病人數:", len(val_pids))

        train_files = []
        val_files = []

        for pid in train_pids:
            train_files.extend(patient_to_files[pid])
        for pid in val_pids:
            val_files.extend(patient_to_files[pid])

        fold_dir = os.path.join(KFOLD_ROOT, f"fold{fold_idx}")
        train_img_dir = os.path.join(fold_dir, "train", "images")
        train_lbl_dir = os.path.join(fold_dir, "train", "labels")
        val_img_dir   = os.path.join(fold_dir, "val", "images")
        val_lbl_dir   = os.path.join(fold_dir, "val", "labels")

        print(f"  Train 影像數: {len(train_files)}")
        print(f"  Val   影像數: {len(val_files)}")

        # 寫入檔案
        write_list_to_yolo_folder(train_files, train_img_dir, train_lbl_dir)
        write_list_to_yolo_folder(val_files,   val_img_dir,   val_lbl_dir)

        # 建立 data.yaml
        data_yaml_path = os.path.join(fold_dir, f"fold{fold_idx}_data.yaml")
        with open(data_yaml_path, "w", encoding="utf-8") as f:
            f.write(f"path: {fold_dir.replace(os.sep, '/')}\n")
            f.write("train: train/images\n")
            f.write("val: val/images\n")
            f.write("names:\n")
            f.write("  0: valve\n")

        print(f"✅ 已建立 {data_yaml_path}")

create_fold_datasets(N_FOLDS, SEED)


NameError: name 'patient_to_files' is not defined

In [3]:
# Cell 7: 訓練單一 fold 的函式

def train_one_fold(fold_idx: int):
    """
    在 yolo_kfold_roi/fold{fold_idx} 上訓練 YOLOv12n 1024x1024 (ROI 版)
    輸出到 runs/detect_kfold_roi/fold{fold_idx}_1024
    """
    fold_dir  = os.path.join(KFOLD_ROOT, f"fold{fold_idx}")
    data_yaml = os.path.join(fold_dir,  f"fold{fold_idx}_data.yaml")

    if not os.path.exists(data_yaml):
        print(f"❌ 找不到 {data_yaml}，請先跑 create_fold_datasets()")
        return

    print(f"\n🚀 開始訓練 Fold {fold_idx}")
    print(f"   使用 data.yaml: {data_yaml}")

    model = YOLO(MODEL_NAME)

    results = model.train(
        data=data_yaml,
        imgsz=IMG_SIZE,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        device=0,          # 單卡 GPU
        workers=0,         # Windows 推薦 0
        optimizer="AdamW",
        lr0=2e-5,
        lrf=0.05,
        patience=25,
        cache="ram",
        amp=True,

        mosaic=0.0,
        copy_paste=0.0,
        auto_augment=None,
        erasing=0.0,
        hsv_h=0.0,
        hsv_s=0.0,
        hsv_v=0.0,
        fliplr=0.5,
        box=4.5,
        cls=2.0,
        dfl=1.5,

        project=PROJECT,
        name=f"fold{fold_idx}_1024_roi",
        exist_ok=True,
        save_period=1,
    )

    print(f"🎉 Fold {fold_idx} 訓練完成，輸出在 {PROJECT}/fold{fold_idx}_1024_roi")
    return results


In [4]:
# Cell 8: 實際執行訓練

# 例：只先跑 fold 5，確定流程通順
train_one_fold(5)

# 如果之後要跑全部：
#for k in range(1, N_FOLDS + 1):
#    train_one_fold(k)


🚀 開始訓練 Fold 5
   使用 data.yaml: C:\Users\User\Desktop\AICUP\yolo_kfold_roi\fold5\fold5_data.yaml
New https://pypi.org/project/ultralytics/8.3.233 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.232  Python-3.13.9 torch-2.9.0+cu130 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=64, bgr=0.0, box=4.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=2.0, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\User\Desktop\AICUP\yolo_kfold_roi\fold5\fold5_data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=45, erasing=0.0, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=2e-05, lrf=0.05, mask_ratio=4, max_det=

KeyboardInterrupt: 